# Tweet sentiment analysis during Covid times

API set-up

In [1]:
# Import libraries
import tweepy
from textblob import TextBlob
from wordcloud import WordCloud
import pandas as pd
import numpy as np
import re
import nltk
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import TweetTokenizer
plt.style.use('fivethirtyeight')

In [2]:
# Twitter API credentials
consumer_key = 'wrrHhdKxgYfKH0WKwOm7Uyylq'
consumer_secret = '9qb3hTNywK5lU5YTIYSxd3ADrloD9xhjdlQJzRwXL4wk3Up4Ar'
access_token = '284537952-dKg8gIk3Chn85c6jB0OLsbMdv7dqOtgKBc5iRONK'
access_token_secret = '6cv20z56cCwUbopvN8PHKqo2bcQIzHD0pShpZxnHDnadH'

In [3]:
# Create authentication object
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth, wait_on_rate_limit=True)
public_tweets = api.home_timeline()

In [4]:
# search_words = "#wildfires"

columns=['text', 'user', 'date', 'source', 'location', 'url']
msgs = []
msg =[]

for tweet in tweepy.Cursor(api.search, q='accutane', rpp=100).items(5000):
    msg = [tweet.text, tweet.user.screen_name, tweet.created_at, tweet.source, tweet.user.location, tweet.source_url] 
    msg = tuple(msg)                    
    msgs.append(msg)

    
df = pd.DataFrame(msgs, columns=columns)

In [38]:
def remove_by_regex(tweet, regexp):
        return re.sub(regexp, '', tweet)

def remove_url(tweet):
    return remove_by_regex(tweet, re.compile(r"http.?://[^\s]+[\s]?"))

def remove_special_char(tweet):
    return re.sub(r"[^a-zA-Z0-9 ]", " ", tweet) #add space placeholder

def remove_numbers(tweet):
    return remove_by_regex(tweet, re.compile(r"[1234567890]"))

def clean_up(tweet):
    tweet = remove_numbers(tweet)
    tweet = remove_url(tweet)
    tweet = remove_special_char(tweet)
    tweet = remove_rt(tweet)
    return tweet.lower().strip()

def remove_rt(tweet):
    return remove_by_regex(tweet, re.compile(r"[Rr][Tt] "))

In [39]:
df['clean_text'] = df['text'].apply(clean_up)

In [7]:
tt = TweetTokenizer()
df['tokenized'] = df['clean_text'].apply(tt.tokenize)

In [8]:
def remove_stopwords(text):
    stopwords_list = stopwords.words('english')
    return [x for x in text if x not in stopwords_list]

In [9]:
df['text_processed'] = df['tokenized'].apply(remove_stopwords)

In [10]:
def stem_list(l):
    stemmer = SnowballStemmer("english")
    stemmed = []
    for i in l:
        stemmed.append(stemmer.stem(i))
    return stemmed

In [11]:
df['text_stemmed'] = df['text_processed'].apply(stem_list)

In [12]:
def stem_lem(l):
    lemmatizer = WordNetLemmatizer()
    lemmatized = []
    for i in l:
        lemmatized.append(lemmatizer.lemmatize(i))
    return lemmatized

In [13]:
df['text_lemmatized'] = df['text_stemmed'].apply(stem_lem)

In [14]:
def sentiment(l):
    sid = SentimentIntensityAnalyzer()
    sentiments = []
    sentiments.append(sid.polarity_scores(l))
    return sentiments

In [15]:
df['sentiment'] = df['clean_text'].apply(lambda x: sentiment(x))

In [16]:
c = list(df.iloc[0]['sentiment'][0].keys())
c

['neg', 'neu', 'pos', 'compound']

In [17]:
s = pd.DataFrame([list(df["sentiment"].apply(lambda x: x[0].values()).loc[i]) for i in range(df.shape[0])], columns = c)

In [18]:
df_s = pd.concat([df,s], axis = 1)

In [19]:
df_s.shape

(880, 16)

In [20]:
df_s[df_s['neg']>=0.2]

,text,user,date,source,location,url,clean_text,tokenized,text_processed,text_stemmed,text_lemmatized,sentiment,neg,neu,pos,compound
4,My sister always working on her nails business...,Kamawai808,2020-12-14 08:31:39,Twitter for iPhone,Nigeria,http://twitter.com/download/iphone,my sister always working on her nails business...,"[my, sister, always, working, on, her, nails, ...","[sister, always, working, nails, business, poo...","[sister, alway, work, nail, busi, poor, babi, ...","[sister, alway, work, nail, busi, poor, babi, ...","[{'neg': 0.284, 'neu': 0.716, 'pos': 0.0, 'com...",0.284,0.716,0.000,-0.7462
6,me signing my life away to accutane knowing I ...,marianne_ns,2020-12-14 06:47:57,Twitter for iPhone,,http://twitter.com/download/iphone,me signing my life away to accutane knowing i ...,"[me, signing, my, life, away, to, accutane, kn...","[signing, life, away, accutane, knowing, sever...","[sign, life, away, accutan, know, sever, back,...","[sign, life, away, accutan, know, sever, back,...","[{'neg': 0.305, 'neu': 0.695, 'pos': 0.0, 'com...",0.305,0.695,0.000,-0.7845
26,#1: Accutane. I had really bad acne growing up...,LeBouji,2020-12-14 01:26:06,Twitter for iPhone,,http://twitter.com/download/iphone,accutane i had really bad acne growing up and...,"[accutane, i, had, really, bad, acne, growing,...","[accutane, really, bad, acne, growing, mother,...","[accutan, realli, bad, acn, grow, mother, work...","[accutan, realli, bad, acn, grow, mother, work...","[{'neg': 0.259, 'neu': 0.674, 'pos': 0.066, 'c...",0.259,0.674,0.066,-0.7737
27,@mapofhao PLSKSKSKSKS I WAS ON ACCUTANE 😭😭😭 do...,mingy3w,2020-12-14 01:21:57,Twitter for iPhone,changbin 🥰,http://twitter.com/download/iphone,mapofhao plsksksksks i was on accutane do ...,"[mapofhao, plsksksksks, i, was, on, accutane, ...","[mapofhao, plsksksksks, accutane, recommend]","[mapofhao, plsksksksks, accutan, recommend]","[mapofhao, plsksksksks, accutan, recommend]","[{'neg': 0.232, 'neu': 0.768, 'pos': 0.0, 'com...",0.232,0.768,0.000,-0.2755
31,Drugs approved by FDA but caused harm:\nAccuta...,cyncynj,2020-12-13 23:39:31,Twitter Web App,"Blue State, Burbs",https://mobile.twitter.com,drugs approved by fda but caused harm accutan...,"[drugs, approved, by, fda, but, caused, harm, ...","[drugs, approved, fda, caused, harm, accutane,...","[drug, approv, fda, caus, harm, accutan, vioxx...","[drug, approv, fda, caus, harm, accutan, vioxx...","[{'neg': 0.269, 'neu': 0.623, 'pos': 0.108, 'c...",0.269,0.623,0.108,-0.5927
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
848,"@Hoenyyyyy Ikr it sucks, if u have severe cons...",yaarwolarka,2020-12-06 02:06:31,Twitter for Android,Isb/Dxb,http://twitter.com/download/android,hoenyyyyy ikr it sucks if u have severe consu...,"[hoenyyyyy, ikr, it, sucks, if, u, have, sever...","[hoenyyyyy, ikr, sucks, u, severe, consult, de...","[hoenyyyyi, ikr, suck, u, sever, consult, derm...","[hoenyyyyi, ikr, suck, u, sever, consult, derm...","[{'neg': 0.264, 'neu': 0.57, 'pos': 0.166, 'co...",0.264,0.570,0.166,-0.2263
860,"6. i started accutane so no, i cannot drink",cheyenneeford,2020-12-05 22:12:08,Twitter for iPhone,"myrtle beach, sc",http://twitter.com/download/iphone,i started accutane so no i cannot drink,"[i, started, accutane, so, no, i, cannot, drink]","[started, accutane, cannot, drink]","[start, accutan, cannot, drink]","[start, accutan, cannot, drink]","[{'neg': 0.364, 'neu': 0.636, 'pos': 0.0, 'com...",0.364,0.636,0.000,-0.4341
865,@GwendolynKansen Yes please. Accutane did its...,ShotzFogelstein,2020-12-05 21:03:09,Twitter for Android,,http://twitter.com/download/android,gwendolynkansen yes please accutane did its ...,"[gwendolynkansen, yes, please, accutane, did, ...","[gwendolynkansen, yes, please, accutane, job, ...","[gwendolynkansen, yes, pleas, accutan, job, co...","[gwendolynkansen, yes, plea, accutan, job, col...","[{'neg': 0.302, 'neu': 0.566, 'pos': 0.132, 'c...",0.302,0.566,0.132,-0.7579
871,Can you get lip filler on accut

In [21]:
df_s[df_s['pos']>=0.5]

,text,user,date,source,location,url,clean_text,tokenized,text_processed,text_stemmed,text_lemmatized,sentiment,neg,neu,pos,compound
51,@salmamamer Accutane is my one true love,janahayaly,2020-12-13 17:34:26,Twitter for iPhone,"Cairo, Egypt",http://twitter.com/download/iphone,salmamamer accutane is my one true love,"[salmamamer, accutane, is, my, one, true, love]","[salmamamer, accutane, one, true, love]","[salmamam, accutan, one, true, love]","[salmamam, accutan, one, true, love]","[{'neg': 0.0, 'neu': 0.417, 'pos': 0.583, 'com...",0.0,0.417,0.583,0.7906
113,🥺❤️ ily! I don’t have a routine anymore I’m ju...,lilbabybrayy,2020-12-12 19:36:15,Twitter for iPhone,"California, USA",http://twitter.com/download/iphone,ily i don t have a routine anymore i m just o...,"[ily, i, don, t, have, a, routine, anymore, i,...","[ily, routine, anymore, accutane, lol]","[ili, routin, anymor, accutan, lol]","[ili, routin, anymor, accutan, lol]","[{'neg': 0.0, 'neu': 0.493, 'pos': 0.507, 'com...",0.0,0.493,0.507,0.8020
120,accutane packets are so funny lol https://t.co...,lacedstrwbrries,2020-12-12 18:47:16,Twitter for iPhone,"San Antonio, TX",http://twitter.com/download/iphone,accutane packets are so funny lol,"[accutane, packets, are, so, funny, lol]","[accutane, packets, funny, lol]","[accutan, packet, funni, lol]","[accutan, packet, funni, lol]","[{'neg': 0.0, 'neu': 0.37, 'pos': 0.63, 'compo...",0.0,0.370,0.630,0.7795
205,@thequeenpri accutane is &gt;&gt;&gt;\n\nwhew! 😻,xiukey_x,2020-12-11 16:33:27,Twitter for iPhone,,http://twitter.com/download/iphone,thequeenpri accutane is gt gt gt whew,"[thequeenpri, accutane, is, gt, gt, gt, whew]","[thequeenpri, accutane, gt, gt, gt, whew]","[thequeenpri, accutan, gt, gt, gt, whew]","[thequeenpri, accutan, gt, gt, gt, whew]","[{'neg': 0.0, 'neu': 0.388, 'pos': 0.612, 'com...",0.0,0.388,0.612,0.6486
239,i love u accutane,yarasaleem_,2020-12-11 04:28:37,Twitter for iPhone,"Dubai, United Arab Emirates",http://twitter.com/download/iphone,i love u accutane,"[i, love, u, accutane]","[love, u, accutane]","[love, u, accutan]","[love, u, accutan]","[{'neg': 0.0, 'neu': 0.192, 'pos': 0.808, 'com...",0.0,0.192,0.808,0.6369
567,@marcusstonio We love to see fellow accutane s...,emulio_,2020-12-08 08:52:39,Twitter for iPhone,"Visalia, CA",http://twitter.com/download/iphone,marcusstonio we love to see fellow accutane su...,"[marcusstonio, we, love, to, see, fellow, accu...","[marcusstonio, love, see, fellow, accutane, su...","[marcusstonio, love, see, fellow, accutan, suc...","[marcusstonio, love, see, fellow, accutan, suc...","[{'neg': 0.0, 'neu': 0.47, 'pos': 0.53, 'compo...",0.0,0.470,0.530,0.8360
596,Thanks Accutane 👍🏽 https://t.co/SiFeHCqU4l,kookoosbun,2020-12-08 04:21:36,Twitter for iPhone,i’m black just so yall know,http://twitter.com/download/iphone,thanks accutane,"[thanks, accutane]","[thanks, accutane]","[thank, accutan]","[thank, accutan]","[{'neg': 0.0, 'neu': 0.256, 'pos': 0.744, 'com...",0.0,0.256,0.744,0.4404
714,Accutane does miracle work 🤩 https://t.co/skza...,HectorEscamil,2020-12-07 02:45:52,Twitter for iPhone,"El Paso, TX",http://twitter.com/download/iphone,accutane does miracle work,"[accutane, does, miracle, work]","[accutane, miracle, work]","[accutan, miracl, work]","[accutan, miracl, work]","[{'neg': 0.0, 'neu': 0.441, 'pos': 0.559, 'com...",0.0,0.441,0.559,0.5859


In [22]:
df_s['pos'].median()

0.0

In [23]:
df_s['neg'].median()

0.0

In [24]:
df_s

,text,user,date,source,location,url,clean_text,tokenized,text_processed,text_stemmed,text_lemmatized,sentiment,neg,neu,pos,compound
0,i just had an appointment with the nurse for a...,judyhaleswife,2020-12-14 10:07:36,Twitter for iPhone,she/her • 21,http://twitter.com/download/iphone,i just had an appointment with the nurse for a...,"[i, just, had, an, appointment, with, the, nur...","[appointment, nurse, check, accutane, apparent...","[appoint, nurs, check, accutan, appar, experie...","[appoint, nurs, check, accutan, appar, experie...","[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compoun...",0.000,1.000,0.000,0.0000
1,"Aku dah start makan accutane pagi tadi, ni pim...",lenatikah,2020-12-14 09:53:05,Twitter for iPhone,"Johor Bahru, Johor",http://twitter.com/download/iphone,aku dah start makan accutane pagi tadi ni pim...,"[aku, dah, start, makan, accutane, pagi, tadi,...","[aku, dah, start, makan, accutane, pagi, tadi,...","[aku, dah, start, makan, accutan, pagi, tadi, ...","[aku, dah, start, makan, accutan, pagi, tadi, ...","[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compoun...",0.000,1.000,0.000,0.0000
2,@biebsidea I’ve never gone to the doctor for a...,bieberstruce,2020-12-14 09:46:56,Twitter for iPhone,"Seattle, WA",http://twitter.com/download/iphone,biebsidea i ve never gone to the doctor for ac...,"[biebsidea, i, ve, never, gone, to, the, docto...","[biebsidea, never, gone, doctor, acne, wanna, ...","[biebsidea, never, gone, doctor, acn, wanna, a...","[biebsidea, never, gone, doctor, acn, wanna, a...","[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compoun...",0.000,1.000,0.000,0.0000
3,effect makan accutane ni memang cemni ke lain ...,wslo_,2020-12-14 09:33:32,Twitter for iPhone,,http://twitter.com/download/iphone,effect makan accutane ni memang cemni ke lain ...,"[effect, makan, accutane, ni, memang, cemni, k...","[effect, makan, accutane, ni, memang, cemni, k...","[effect, makan, accutan, ni, memang, cemni, ke...","[effect, makan, accutan, ni, memang, cemni, ke...","[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compoun...",0.000,1.000,0.000,0.0000
4,My sister always working on her nails business...,Kamawai808,2020-12-14 08:31:39,Twitter for iPhone,Nigeria,http://twitter.com/download/iphone,my sister always working on her nails business...,"[my, sister, always, working, on, her, nails, ...","[sister, always, working, nails, business, poo...","[sister, alway, work, nail, busi, poor, babi, ...","[sister, alway, work, nail, busi, poor, babi, ...","[{'neg': 0.284, 'neu': 0.716, 'pos': 0.0, 'com...",0.284,0.716,0.000,-0.7462
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
875,I got told the Accutane I’m taking might have ...,champagne_tears,2020-12-05 19:22:13,Twitter for iPhone,,http://twitter.com/download/iphone,i got told the accutane i m taking might have ...,"[i, got, told, the, accutane, i, m, taking, mi...","[got, told, accutane, taking, might, heightene...","[got, told, accutan, take, might, heighten, bo...","[got, told, accutan, take, might, heighten, bo...","[{'neg': 0.26, 'neu': 0.634, 'pos': 0.106, 'co...",0.260,0.634,0.106,-0.5667
876,@abrilluisarios I watched ur yt video a while ...,allyanjumm,2020-12-05 19:21:20,Twitter for iPhone,"Austin, TX",http://twitter.com/download/iphone,abrilluisarios i watched ur yt video a while b...,"[abrilluisarios, i, watched, ur, yt, video, a,...","[abrilluisarios, watched, ur, yt, video, back,...","[abrilluisario, watch, ur, yt, video, back, u,...","[abrilluisario, watch, ur, yt, video, back, u,...","[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compoun...",0.000,1.000,0.000,0.0000
877,accutane journey starts monday 🙌🏼,_skylarcardia,2020-12-05 19:20:07,Twitter for iPhone,,http://twitter.com/download/iphone,accutane journey starts monday,"[accutane, journey, starts, monday]","[accutane, journey, starts, monday]","[accutan, journey, start, monday]","[accutan, journey, start, monday]","[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compoun...",0.000,1.000,0.000,0.0000
878,guess the accutane worked h

In [25]:
all_words = []

for index, value in df_s.text_lemmatized.iteritems():
    if value not in all_words:
        all_words += value

top_features = [x[0] for x in nltk.FreqDist(all_words).most_common(20)]

In [26]:
top_features

['accutan',
 'skin',
 'rt',
 'acn',
 'day',
 'read',
 'one',
 'get',
 'year',
 'start',
 'go',
 'take',
 'month',
 'last',
 'fuck',
 'urdadssidepiec',
 'dri',
 'would',
 'like',
 'today']

In [27]:
def build_features(words):
    features = {}
    for w in top_features:
        features[w] = (1 if w in words else 0)
    return features

In [28]:
featuresets = []

for index, row in df_s.iterrows():
    featuresets.append((build_features(row['text_lemmatized'])))

In [29]:
featuresets[:10]

[{'accutan': 1,
  'skin': 0,
  'rt': 0,
  'acn': 0,
  'day': 0,
  'read': 0,
  'one': 0,
  'get': 0,
  'year': 0,
  'start': 0,
  'go': 0,
  'take': 0,
  'month': 0,
  'last': 0,
  'fuck': 0,
  'urdadssidepiec': 0,
  'dri': 0,
  'would': 0,
  'like': 0,
  'today': 0},
 {'accutan': 1,
  'skin': 0,
  'rt': 0,
  'acn': 0,
  'day': 0,
  'read': 0,
  'one': 0,
  'get': 0,
  'year': 0,
  'start': 1,
  'go': 0,
  'take': 0,
  'month': 0,
  'last': 0,
  'fuck': 0,
  'urdadssidepiec': 0,
  'dri': 0,
  'would': 0,
  'like': 0,
  'today': 0},
 {'accutan': 1,
  'skin': 0,
  'rt': 0,
  'acn': 1,
  'day': 0,
  'read': 0,
  'one': 0,
  'get': 0,
  'year': 0,
  'start': 0,
  'go': 0,
  'take': 1,
  'month': 0,
  'last': 0,
  'fuck': 0,
  'urdadssidepiec': 0,
  'dri': 0,
  'would': 0,
  'like': 0,
  'today': 0},
 {'accutan': 1,
  'skin': 0,
  'rt': 0,
  'acn': 0,
  'day': 0,
  'read': 0,
  'one': 0,
  'get': 0,
  'year': 0,
  'start': 0,
  'go': 0,
  'take': 0,
  'month': 0,
  'last': 0,
  'fuck': 0,
 

In [30]:
featuresets

[{'accutan': 1,
  'skin': 0,
  'rt': 0,
  'acn': 0,
  'day': 0,
  'read': 0,
  'one': 0,
  'get': 0,
  'year': 0,
  'start': 0,
  'go': 0,
  'take': 0,
  'month': 0,
  'last': 0,
  'fuck': 0,
  'urdadssidepiec': 0,
  'dri': 0,
  'would': 0,
  'like': 0,
  'today': 0},
 {'accutan': 1,
  'skin': 0,
  'rt': 0,
  'acn': 0,
  'day': 0,
  'read': 0,
  'one': 0,
  'get': 0,
  'year': 0,
  'start': 1,
  'go': 0,
  'take': 0,
  'month': 0,
  'last': 0,
  'fuck': 0,
  'urdadssidepiec': 0,
  'dri': 0,
  'would': 0,
  'like': 0,
  'today': 0},
 {'accutan': 1,
  'skin': 0,
  'rt': 0,
  'acn': 1,
  'day': 0,
  'read': 0,
  'one': 0,
  'get': 0,
  'year': 0,
  'start': 0,
  'go': 0,
  'take': 1,
  'month': 0,
  'last': 0,
  'fuck': 0,
  'urdadssidepiec': 0,
  'dri': 0,
  'would': 0,
  'like': 0,
  'today': 0},
 {'accutan': 1,
  'skin': 0,
  'rt': 0,
  'acn': 0,
  'day': 0,
  'read': 0,
  'one': 0,
  'get': 0,
  'year': 0,
  'start': 0,
  'go': 0,
  'take': 0,
  'month': 0,
  'last': 0,
  'fuck': 0,
 

In [31]:
features = pd.DataFrame(featuresets)

In [32]:
features.sum().sort_values(ascending=False)

accutan           759
skin              109
rt                106
acn                83
one                63
day                60
year               58
get                58
go                 55
urdadssidepiec     53
fuck               52
last               52
take               52
start              51
month              51
would              50
dri                46
like               45
today              45
read               38
dtype: int64

In [ ]:
# Creating and generate a word cloud image:

for i in tqdm(range(100)):
    wordcloud = WordCloud(max_font_size=50, max_words=100, background_color=“white”, colormap=“”).generate(negative_text)
# Displaying the image:
plt.figure(figsize=(7,5))
plt.imshow(wordcloud, interpolation=‘bilinear’)
plt.axis(“off”)